## 0. 라이브러리 호출
1. `import os`: 운영 체제와 상호 작용합니다.
2. `import logging`: 로깅 기능을 사용합니다.
3. `from gtts import gTTS`: Google 텍스트 음성 변환(TTS) 기능을 사용합니다.
4. `import pdfplumber`: PDF 파일 처리 기능을 사용합니다.
5. `from langchain.vectorstores import Chroma`: 텍스트 분석을 위한 Chroma 벡터 스토어를 사용합니다.
6. `from langchain.embeddings import OpenAIEmbeddings`: OpenAI 임베딩을 사용하여 텍스트를 임베딩합니다.
7. `from langchain.text_splitter import RecursiveCharacterTextSplitter`: RecursiveCharacterTextSplitter를 사용하여 텍스트를 분할합니다.
8. `from langchain.chains import RetrievalQA`: 검색 기반의 질문 응답 시스템을 사용합니다.
9. `from langchain.chat_models import ChatOpenAI`: 대화 모델을 사용합니다.
10. `from langchain.docstore.document import Document`: 문서 처리를 위한 Document 모듈을 사용합니다.
11. `from langchain_core.prompts import ChatPromptTemplate`: 대화 프롬프트 템플릿을 사용합니다.
12. `from langchain.chains import LLMChain`: 대형 언어 모델 (LLM) 체인을 사용합니다.
13. `import speech_recognition as sr`: 음성 인식 기능을 사용합니다.
14. `import tkinter as tk`: GUI 생성을 위해 Tkinter를 사용합니다.
15. `from tkinter import messagebox`: Tkinter의 메시지 박스를 사용합니다.
16. `from playsound import playsound`: 사운드를 재생합니다.
17. `from pydub import AudioSegment`: 오디오 파일을 처리하기 위해 Pydub의 AudioSegment를 사용합니다.
18. `from pydub.playback import play`: Pydub을 사용하여 오디오 파일을 재생합니다.
19. `import googleapiclient.discovery`: Google API를 사용하기 위해 discovery를 가져옵니다.
20. `from pytube import YouTube`: YouTube 비디오 다운로드를 위해 Pytube를 사용합니다.
21. `import io`: 입출력 기능을 사용합니다.


In [1]:
import os
import logging
import threading
from gtts import gTTS
import pdfplumber
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import speech_recognition as sr
import tkinter as tk
from tkinter import messagebox
from playsound import playsound
from pydub import AudioSegment
from pydub.playback import play
import googleapiclient.discovery
from pytube import YouTube
import io

## 1. 환경변수 설정

이 코드는 로그 파일을 설정하고 쿼리와 응답을 기록하는 함수를 정의합니다. 아래는 코드의 주요 구성 요소에 대한 설명입니다:

1. `os.environ`을 사용하여 환경 변수를 설정합니다. 여기서는 OpenAI API 및 Google API 키를 설정하고 있습니다.

2. `log_directory`, `log_filename`, `log_path` 변수들은 로그 파일의 경로 및 이름을 설정하는 데 사용됩니다.

3. `os.path.exists(log_directory)`를 통해 로그 디렉토리가 존재하는지 확인하고, 존재하지 않으면 디렉토리를 생성합니다.

4. `logging.basicConfig` 함수를 사용하여 기본 로그 설정을 수행합니다. 이 함수를 통해 로그 파일의 경로, 로그 레벨, 로그 포맷 등을 설정합니다.

5. `log_query_and_response` 함수는 두 개의 인자 `query`와 `response`를 받아 로그에 기록합니다. 각각의 쿼리와 응답은 시간 정보와 함께 기록됩니다.

이 코드는 주로 쿼리와 응답을 로그 파일에 기록하여 추후 디버깅 및 분석에 활용될 수 있도록 합니다.



In [2]:
# 1. 환경변수 설정
os.environ["OPENAI_API_KEY"] = os.environ.get('OPENAI_API_KEY')
os.environ["GOOGLE_API_KEY"] = os.environ.get('GOOGLE_API_KEY')

log_directory = "../log/"
log_filename = "query_log.txt"
log_path = os.path.join(log_directory, log_filename)

if not os.path.exists(log_directory):
    os.makedirs(log_directory)

logging.basicConfig(filename=log_path,
                    level=logging.INFO,
                    format='%(asctime)s - %(message)s',
                    filemode='a')

def log_query_and_response(query, response):
    logging.info(f"Query: {query}")
    logging.info(f"Response: {response}")

## 2. PDF 텍스트 및 테이블 처리
이 코드는 PDF 파일에서 테이블을 추출하고 처리하는 함수들을 정의합니다. 아래는 코드의 주요 구성 요소에 대한 설명입니다:

1. `extract_tables_from_pdfplumber(pdf_path)`: 이 함수는 `pdfplumber`를 사용하여 주어진 PDF 파일에서 테이블을 추출합니다. PDF 파일의 각 페이지를 반복하면서 `extract_tables()` 메서드를 사용하여 테이블을 추출하고, 각 테이블을 `tables_content` 리스트에 추가합니다. 마지막으로 추출된 모든 테이블을 반환합니다.

2. `process_pdfs(directory)`: 이 함수는 주어진 디렉토리 내의 모든 PDF 파일을 처리합니다. 먼저 `os.listdir(directory)`를 사용하여 디렉토리 내의 모든 파일을 반복하고, `.pdf` 확장자를 가진 파일만 선택합니다. 선택된 각 PDF 파일에 대해 `extract_tables_from_pdfplumber()` 함수를 호출하여 테이블을 추출하고, 추출된 테이블 데이터를 `documents` 리스트에 추가합니다. 각 테이블 데이터는 `Document` 클래스로 래핑되어 저장되는데, 이 클래스는 페이지 내용과 메타데이터를 저장합니다. 처리된 모든 문서를 반환합니다.

이 코드는 PDF 파일에서 테이블을 추출하고 이를 적절히 처리하여 다른 작업에 활용할 수 있도록 합니다.


In [3]:
# 2. PDF 텍스트 및 테이블 처리
def extract_tables_from_pdfplumber(pdf_path):
    tables_content = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                tables_content.append(table)
    return tables_content

def process_pdfs(directory):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory, filename)
            tables_content = extract_tables_from_pdfplumber(file_path)
            for table_data in tables_content:
                table_text = '\n'.join(['\t'.join([str(cell) if cell is not None else '' for cell in row]) for row in table_data])
                documents.append(Document(page_content=table_text, metadata={"source": filename}))
    return documents

## 3. ChromaDB로 벡터스토어 생성 또는 호출
이 코드는 벡터 스토어를 생성하거나 로드하는 과정을 구현합니다. 아래는 코드의 주요 구성 요소에 대한 설명입니다:
1. `persist_directory`: 벡터 스토어를 저장하거나 로드하는 데 사용되는 디렉토리 경로를 설정합니다.
2. 디렉토리가 존재하지 않으면:
    - PDF 파일을 처리하여 문서를 생성합니다. `process_pdfs()` 함수를 사용하여 PDF 파일을 처리하고 문서를 생성합니다.
    - 문서를 청크로 나누기 위해 `RecursiveCharacterTextSplitter`를 초기화합니다. 이를 사용하여 문서를 적절한 크기의 청크로 나눕니다.
    - 벡터 스토어를 생성하기 위해 임베딩과 `Chroma` 클래스를 사용합니다. `Chroma.from_documents()` 메서드를 사용하여 문서를 기반으로 벡터 스토어를 생성합니다. 생성된 벡터 스토어는 지정된 `persist_directory`에 저장됩니다.
3. 디렉토리가 이미 존재하면:
    - 저장된 벡터 스토어를 로드합니다. `Chroma` 클래스를 사용하여 저장된 벡터 스토어를 로드합니다. 로드할 때 사용되는 벡터 스토어의 디렉토리 경로와 임베딩 함수를 설정합니다.
이 코드는 PDF 파일을 처리하여 벡터 스토어를 생성하고, 이를 디스크에 저장하여 재사용합니다. 만약 이미 생성된 벡터 스토어가 있다면, 디스크에서 로드하여 중복 계산을 피합니다.


In [4]:
# 3. ChromaDB로 벡터스토어 생성 또는 호출
persist_directory = "../car_info_sql/"

if not os.path.exists(persist_directory):
    documents = process_pdfs('../pdfs/')

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    embedding = OpenAIEmbeddings()
    vectordb = Chroma.from_documents(documents=texts,
                                     embedding=embedding,
                                     persist_directory=persist_directory)
    vectordb.persist()
else:
    vectordb = Chroma(persist_directory=persist_directory,
                      embedding_function=OpenAIEmbeddings())

/opt/anaconda3/envs/CHAT_BOT_SERVICE/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


## 4. OPEN AI API 체이닝
위 코드는 다음과 같은 작업을 수행합니다:

1. **RetrievalQA 체인 생성**:
   - `retriever = vectordb.as_retriever(search_kwargs={"k": 3})`: 벡터 데이터베이스에서 RetrievalQA 체인을 생성합니다. `search_kwargs`는 검색 옵션을 설정합니다. 이 경우, 가장 가까운 3개의 벡터를 검색합니다.

2. **프롬프트 템플릿 초기화**:
   - 시스템과 사용자 간의 대화를 위한 프롬프트 템플릿을 초기화합니다. 시스템은 차량 정보를 제공하고 음악 재생도 지원한다는 안내를 포함합니다.

3. **GPT-4o 모델로 채팅 모델 초기화**:
   - `llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=2048)`: GPT-4o 모델을 사용하여 대화 모델을 초기화합니다. 토큰 최대 개수는 최대로 설정되어 있습니다.

4. **언어 모델과 프롬프트 템플릿을 포함한 체인 초기화**:
   - `chain = LLMChain(llm=llm, prompt=chat_prompt, output_key='result')`: 언어 모델과 프롬프트 템플릿을 포함한 체인을 초기화합니다. 출력 키는 'result'로 설정되어 있습니다.

5. **RetrievalQA 체인 생성**:
   - `qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)`: RetrievalQA 체인을 생성합니다. 이때, 문서 정보를 반환하는 옵션이 추가되었습니다.


In [5]:
# 4. OPEN AI API 체이닝
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "안녕하세요! 차량 정보를 정확히 제공해드릴 수 있는 시스템입니다."
               "또한, 음악 재생을 요청하면 음악 재생도 해드릴 수 있어요"
               "현재 데이터베이스에 '기아 K3'와 '폭스바겐 골프 7세대'의 정보가 있습니다."
               "질문하실 때 특정 차량을 정확하게 구분해주세요."
               "데이터는 영어와 한국어로 구성되어 있으며, 답변은 한국어로 드립니다."
               "길이 단위는 기본적으로 미터(m)로 사용합니다."
               "데이터에 마일(mile)이 있을 경우 미터로 환산해 드릴게요."
               "차량 정보를 혼동하지 않도록 주의할게요."
               "상세한 설명을 요청하셔도 답변은 250자 이내로 간결하게 드립니다."
               "사람과 대화하듯 편하게 구어체로 답변해 주되 존댓말로 답변해 주세요."
               ),
    ("user", "{user_input}"),
])

llm = ChatOpenAI(model_name="gpt-4o",
                 temperature=0,
                 max_tokens=2048)

chain = LLMChain(llm=llm, prompt=chat_prompt, output_key='result')

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

/opt/anaconda3/envs/CHAT_BOT_SERVICE/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## 5. 텍스트 처리
다음은 위 코드의 설명입니다:

1. **텍스트 변환 함수 (`convert_text(text)`)**:
   - 주어진 텍스트에서 특정 문자열을 다른 문자열로 대체하여 변환하는 함수입니다. 예를 들어, "L"을 "리터"로, "kg.m"을 "킬로그램퍼미터"로, "cc"를 "씨씨"로 등으로 변환합니다.

2. **음성 출력 함수 (`text_to_speech(text, lang='ko')`)**:
   - 주어진 텍스트를 음성으로 변환하여 출력하는 함수입니다.
   - `cleaned_text` 변수는 특수 문자("*" 및 "**")를 제거한 후 변환 함수인 `convert_text()`를 통해 텍스트를 변환합니다.
   - 변환된 텍스트를 기반으로 gTTS(Google Text-to-Speech)를 사용하여 음성을 생성하고, "output.mp3" 파일로 저장한 후 재생합니다.


In [6]:
# 5. 텍스트 처리
def convert_text(text):
    text = text.replace("L", "리터")
    text = text.replace("kg.m", "킬로그램퍼미터")
    text = text.replace("cc", "씨씨")
    text = text.replace("kg", "킬로그램")
    text = text.replace("km", "킬로미터")
    text = text.replace("rpm", "알피엠")
    return text

def text_to_speech(text, lang='ko'):
    cleaned_text = text.replace('*', '').replace('**', '')
    cleaned_text = convert_text(cleaned_text)
    tts = gTTS(text=cleaned_text, lang=lang)
    tts.save("../output/output.mp3")
    playsound("../output/output.mp3")

## 6. 음성 관련 처리
다음은 위 코드의 설명입니다:

1. **음성 인식 및 음원 초기화**:
   - `ping_sound_path = "./sounds/ping-82822.mp3"`: "ping" 사운드 파일의 경로를 지정합니다.
   - `ding_sound_path = "./sounds/ding-36029.mp3"`: "ding" 사운드 파일의 경로를 지정합니다.

2. **음원 재생 함수 (`play_sound(sound_path)`)**:
   - 주어진 사운드 파일 경로에서 사운드를 재생하는 함수입니다.


In [7]:
# 6. 음성 관련 처리
ping_sound_path = "../sounds/ping-82822.mp3"
ding_sound_path = "../sounds/ding-36029.mp3"

def play_sound(sound_path):
    playsound(sound_path)

## 7. 응답 및 쿼리 처리
다음은 위 코드의 설명입니다:

1. **응답 처리 함수 (`process_llm_response(llm_response, query)`)**:
   - 주어진 LLM(대형 언어 모델)의 응답을 처리하고 반환합니다.
   - 만약 응답이 문자열인 경우, 특수 문자("*" 및 "**")를 제거하고 텍스트 변환을 적용합니다. 그리고 "틀어 줘" 또는 "재생"이 쿼리에 포함되어 있으면 음악을 재생하는 함수를 호출합니다.
   - 응답이 딕셔너리인 경우, "result" 키를 통해 응답을 가져온 후 마찬가지로 특수 문자 제거와 텍스트 변환을 적용합니다. 또한, "source_documents" 키를 통해 소스 문서 정보를 출력합니다.

2. **쿼리 및 요약 함수 (`query_and_summarize(query)`)**:
   - 주어진 쿼리를 처리하고 요약한 결과를 반환합니다.
   - 입력으로는 사용자 쿼리를 받고, 이를 체인에 전달하여 응답을 받습니다.
   - 받은 응답을 처리하고 정제한 후에는 로깅 함수를 사용하여 쿼리와 응답을 기록합니다.
   - 마지막으로, 정제된 응답을 음성으로 변환하여 출력하고 반환합니다.


In [8]:
# 7. 응답 및 쿼리 처리
def process_llm_response(llm_response, query):
    if isinstance(llm_response, str):
        cleaned_response = llm_response.replace('*', '').replace('**', '')
        cleaned_response = convert_text(cleaned_response)
        print(cleaned_response)
        if "틀어 줘" in query or "재생" in query:
            keyword = llm_response.split(" ")[0]
            play_music(keyword)
    else:
        cleaned_response = llm_response.get("result", "").replace('*', '').replace('**', '')
        cleaned_response = convert_text(cleaned_response)
        print(cleaned_response)
        print('\n\nSources:')
        for source in llm_response.get("source_documents", []):
            print(source.metadata.get('source', 'Unknown'))
    return cleaned_response

def query_and_summarize(query):
    inputs = {"user_input": query}
    llm_response = chain.invoke(inputs)
    cleaned_response = process_llm_response(llm_response, query)
    log_query_and_response(query, cleaned_response)
    text_to_speech(cleaned_response)
    return cleaned_response

## 8. Youtube Data API 클라이언트 생성
다음은 위 코드의 설명입니다:

**YouTube Data API 클라이언트 생성**:
- `youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=os.getenv("GOOGLE_API_KEY"))`: YouTube Data API 클라이언트를 생성합니다.
  - `"youtube"`: API 서비스의 이름을 지정합니다.
  - `"v3"`: 사용할 API의 버전을 지정합니다.
  - `developerKey=os.getenv("GOOGLE_API_KEY")`: 환경 변수에서 가져온 Google API 키를 사용하여 클라이언트를 빌드합니다.


In [9]:
# 8. Youtube Data API 클라이언트 생성
youtube = googleapiclient.discovery.build("youtube", "v3",
                                          developerKey=os.getenv("GOOGLE_API_KEY"))

## 9. 음악 검색 및 재생

### 코드 설명
음악 검색 및 재생 기능을 구현하는 코드입니다. 이 코드는 유튜브에서 음악을 검색하고, 다운로드한 후 재생합니다. 

#### 1. 음악 재생 스레드와 중지 이벤트 설정
- `music_thread`: 음악 재생을 위한 스레드 객체를 초기화합니다.
- `stop_event`: 음악 재생을 중지하기 위한 이벤트 객체를 초기화합니다.

#### 2. 음악 검색 함수
- `search_music(keyword)`: 유튜브에서 주어진 키워드로 음악을 검색하여 첫 번째 비디오의 ID를 반환합니다.

#### 3. 오디오 다운로드 함수
- `download_audio(video_id)`: 비디오 ID를 사용하여 유튜브 비디오의 오디오 스트림을 다운로드하고, 이를 메모리 버퍼에 저장하여 반환합니다.

#### 4. 음악 재생 스레드 함수
- `play_music_thread(audio_data)`: 주어진 오디오 데이터를 재생하는 스레드 함수입니다. `stop_event`가 설정되기 전까지 오디오를 재생합니다.

#### 5. 음악 재생 함수
- `play_music(keyword)`: 키워드를 사용하여 음악을 검색하고, 다운로드한 후 재생 스레드를 시작합니다.

#### 6. 음악 재생 요청 함수
- `play_music_request(keyword)`: 사용자가 음악 재생을 요청하면, 음악을 검색하고 다운로드한 후 재생 스레드를 시작합니다. 오류가 발생하면 사용자에게 알립니다.

#### 7. 음악 중지 함수
- `stop_music()`: 현재 재생 중인 음악이 있으면 재생을 중지하고, 사용자에게 알립니다. 재생 중인 음악이 없으면 정보 메시지를 표시합니다.

In [10]:
# 9. 음악 검색 및 재생
music_thread = None
stop_event = threading.Event()

def search_music(keyword):
    search_response = youtube.search().list(
        q=keyword,
        part="id",
        type="video",
        maxResults=1
    ).execute()
    
    video_id = search_response["items"][0]["id"]["videoId"]
    return video_id

def download_audio(video_id):
    yt = YouTube(f"https://www.youtube.com/watch?v={video_id}")
    stream = yt.streams.filter(only_audio=True).first()
    audio_data = io.BytesIO()
    stream.stream_to_buffer(audio_data)
    audio_data.seek(0)
    return audio_data

def play_music_thread(audio_data):
    audio = AudioSegment.from_file(audio_data)
    stop_event.clear()
    play(audio)

def play_music(keyword):
    global music_thread
    video_id = search_music(keyword)
    audio_data = download_audio(video_id)
    music_thread = threading.Thread(target=play_music_thread, args=(audio_data,))
    music_thread.start()

def play_music_request(keyword):
    try:
        video_id = search_music(keyword)
        text_to_speech(f"음악을 검색했습니다. '{keyword}'를 유튜브에서 찾아볼게요.")
        audio_data = download_audio(video_id)
        global music_thread
        music_thread = threading.Thread(target=play_music_thread, args=(audio_data,))
        music_thread.start()
    except Exception as e:
        text_to_speech("음악을 재생하는 중에 문제가 발생했습니다. 다시 시도해주세요.")
        print(e)

def stop_music():
    global stop_event, music_thread
    if music_thread and music_thread.is_alive():
        stop_event.set()
        music_thread.join()
        text_to_speech("음악 재생을 중지했습니다.")
    else:
        messagebox.showinfo("Info", "현재 재생 중인 음악이 없습니다.")

## 10. 음성 인식 및 처리

### 코드 설명
이 섹션의 코드는 음성 인식을 통해 사용자의 음성 명령을 처리하고, 이를 기반으로 음악을 재생하거나 질의에 대한 응답을 제공합니다.

#### 1. 초기 설정
- `recognizer = sr.Recognizer()`: 음성 인식을 위한 `Recognizer` 객체를 초기화합니다.
- `is_listening = False`: 음성 인식 상태를 추적하기 위한 플래그를 초기화합니다.

#### 2. 음성 인식 실행 제어
- `if is_listening`: 음성 인식이 이미 실행 중인 경우, 사용자에게 알림을 표시합니다.

#### 3. 음성 인식 준비
- `with sr.Microphone() as source`: 마이크를 사용하여 음성을 입력받기 위한 준비를 합니다.
- `recognizer.adjust_for_ambient_noise(source, duration=1)`: 주변 소음 수준을 조정합니다.
- `text_to_speech("말씀하세요")`: 사용자에게 음성 입력을 요청합니다.
- `play_sound(ping_sound_path)`: 음성 입력이 시작됨을 알리는 소리를 재생합니다.

#### 4. 음성 인식 및 텍스트 처리
- `audio_data = recognizer.listen(source, timeout=5, phrase_time_limit=5)`: 음성을 입력받아 오디오 데이터를 생성합니다.
- `text = recognizer.recognize_google(audio_data, language='ko-KR')`: Google 음성 인식 API를 사용하여 오디오 데이터를 텍스트로 변환합니다.

#### 5. 음악 명령 처리
- `music_synonyms`: 음악 재생 명령어에 대한 다양한 표현을 포함하는 리스트입니다.
- `if any(synonym in text for synonym in music_synonyms)`: 인식된 텍스트가 음악 명령어를 포함하는지 확인합니다.
- `play_music_request(keyword)`: 음악을 검색하고 재생합니다.

#### 6. 일반 질의 처리
- `response_text = query_and_summarize(text)`: 인식된 텍스트를 처리하여 응답을 생성합니다.

#### 7. 예외 처리
- `except sr.WaitTimeoutError`: 음성 인식 타임아웃 예외를 처리합니다.
- `except Exception as e`: 기타 예외를 처리하고 사용자에게 오류를 알립니다.

#### 8. 종료 처리
- `finally: is_listening = False`: 음성 인식이 완료되면 플래그를 초기화합니다.

In [11]:
# 10. 음성 인식 및 처리
recognizer = sr.Recognizer()
is_listening = False

def listen_and_process():
    global is_listening
    if is_listening:
        messagebox.showinfo("Info", "이미 음성 인식이 실행 중입니다.")
        return

    is_listening = True
    try:
        with sr.Microphone() as source:
            print("주변 소음 수준 조정 중...")
            recognizer.adjust_for_ambient_noise(source, duration=1)
            print("소음 조정 완료")

            print("말씀하세요...")
            text_to_speech("말씀하세요")
            play_sound(ping_sound_path)
            audio_data = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            print("음성 인식 중...")
            text = recognizer.recognize_google(audio_data, language='ko-KR')
            print("인식된 텍스트: " + text)
            
            music_synonyms = ["음악 재생", "틀어 줘", "틀어줘", "들려줘", "들려 줘", "플레이 해줘", "재생 해줘", "플레이", "재생", "틀어", "들려"]
            if any(synonym in text for synonym in music_synonyms):
                for synonym in music_synonyms:
                    if synonym in text:
                        keyword = text.replace(synonym, "").strip()
                        response_text = f"음악을 검색했습니다. '{keyword}'를 유튜브에서 찾아볼게요."
                        print(response_text)
                        play_music_request(keyword)
                        is_listening = False
                        break
            else:
                response_text = query_and_summarize(text)
                print(response_text)
    except sr.WaitTimeoutError:
        pass
    except Exception as e:
        messagebox.showerror("Error", f"음성 인식 중 오류가 발생했습니다: {e}")
    finally:
        is_listening = False

## 11. 음성 인식 취소
다음은 위 코드의 설명입니다:

**음성 인식 취소 함수 (`stop_listening()`)**:
- 현재 실행 중인 음성 인식을 취소하는 함수입니다.
- `is_listening` 변수를 사용하여 현재 음성 인식이 실행 중인지를 확인합니다.
- 실행 중인 경우, `is_listening` 변수를 False로 설정하여 음성 인식을 종료하고, 사용자에게 취소되었음을 알리는 정보 메시지를 표시합니다.
- 실행 중이 아닌 경우, 현재 실행 중인 음성 인식이 없다는 정보 메시지를 표시합니다.


In [12]:
# 11. 음성 인식 취소
def stop_listening():
    global is_listening
    if is_listening:
        is_listening = False
        messagebox.showinfo("Info", "음성 인식이 취소되었습니다.")
    else:
        messagebox.showinfo("Info", "현재 실행 중인 음성 인식이 없습니다.")

## 12. Tkinter 사용한 GUI 설정

### 코드 설명
이 섹션의 코드는 Tkinter를 사용하여 GUI를 설정하고, 음성 인식 및 음악 재생 기능을 제공합니다. 

#### 1. Tkinter 윈도우 설정
- `root = tk.Tk()`: Tkinter의 메인 윈도우 객체를 생성합니다.
- `root.title("음성 인식 차량 정보 시스템")`: 윈도우의 제목을 설정합니다.

#### 2. 프레임 생성 및 배치
- `frame = tk.Frame(root)`: 메인 윈도우에 프레임을 생성합니다.
- `frame.pack(pady=20)`: 프레임을 윈도우에 배치하고, 상하단에 20 픽셀의 패딩을 추가합니다.

#### 3. 버튼 생성 및 배치
- `start_button = tk.Button(frame, text="음성 인식 시작", command=listen_and_process)`: 음성 인식을 시작하는 버튼을 생성하고, 클릭 시 `listen_and_process` 함수를 호출합니다.
- `start_button.pack(side=tk.LEFT, padx=10)`: 버튼을 프레임의 왼쪽에 배치하고, 좌우에 10 픽셀의 패딩을 추가합니다.

- `stop_button = tk.Button(frame, text="음악 중지", command=stop_music)`: 음악 재생을 중지하는 버튼을 생성하고, 클릭 시 `stop_music` 함수를 호출합니다.
- `stop_button.pack(side=tk.LEFT, padx=10)`: 버튼을 프레임의 왼쪽에 배치하고, 좌우에 10 픽셀의 패딩을 추가합니다.

- `close_button = tk.Button(frame, text="종료", command=root.quit)`: 프로그램을 종료하는 버튼을 생성하고, 클릭 시 `root.quit` 함수를 호출합니다.
- `close_button.pack(side=tk.LEFT, padx=10)`: 버튼을 프레임의 왼쪽에 배치하고, 좌우에 10 픽셀의 패딩을 추가합니다.

#### 4. Tkinter 이벤트 루프 시작
- `root.mainloop()`: Tkinter의 이벤트 루프를 시작하여 GUI가 사용자 입력을 대기하고 처리할 수 있도록 합니다.

In [13]:
# 12. Tkinter 사용한 GUI 설정
root = tk.Tk()
root.title("음성 인식 차량 정보 시스템")

frame = tk.Frame(root)
frame.pack(pady=20)

start_button = tk.Button(frame, text="음성 인식 시작", command=listen_and_process)
start_button.pack(side=tk.LEFT, padx=10)

stop_button = tk.Button(frame, text="음악 중지", command=stop_music)
stop_button.pack(side=tk.LEFT, padx=10)

close_button = tk.Button(frame, text="종료", command=root.quit)
close_button.pack(side=tk.LEFT, padx=10)

root.mainloop()

주변 소음 수준 조정 중...
소음 조정 완료
말씀하세요...
음성 인식 중...
인식된 텍스트: 컬투쇼 레전드 사연 틀어 줘
음악을 검색했습니다. '컬투쇼 레전드 사연'를 유튜브에서 찾아볼게요.
